BloomTech Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

## Assignment

Complete these tasks for your assignment/project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. Review requirements for your portfolio project on Canvas, and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

Target  = SALE PRICE


My problem is regression


In [58]:
data = "C:/Users/ISHMO_CT/Downloads/Bloomtech/NW_C_price/dataset/NYC_Citywide_Rolling_Calendar_Sales_20240306.csv"

In [59]:
import pandas as pd
import numpy as np

In [60]:
# from ydata_profiling import ProfileReport
# report = ProfileReport(df)
# report.to_notebook_iframe()

In [61]:
# # Finding out if there are outliers
# import matplotlib.pyplot as plt
# plt.scatter(df['GROSS SQUARE FEET'], df['SALE PRICE'])
# plt.xlabel('GSF')
# plt.ylabel('Price')
# plt.show()

### Wrangle and analyse data

In [62]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    df["SALE DATE"] = pd.to_datetime(df["SALE DATE"])
    df.set_index('SALE DATE', inplace=True)
    return df

In [63]:
df = wrangle(data)
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2023-04-18,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,4453 PARK AVENUE,NaN,10457.0,1.0,0.0,1.0,"1,646",1497.0,1899.0,1,A1,215000
2023-08-23,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,4453 PARK AVENUE,NaN,10457.0,1.0,0.0,1.0,"1,646",1497.0,1899.0,1,A1,570000
2023-04-14,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3035,52,NaN,A1,461 EAST 178 STREET,NaN,10457.0,1.0,0.0,1.0,"1,782",1548.0,1899.0,1,A1,0
2023-10-24,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3053,86,NaN,S0,2364 WASHINGTON AVENUE,NaN,10458.0,1.0,2.0,3.0,"1,911",4080.0,1931.0,1,S0,0
2023-06-26,2,BATHGATE,02 TWO FAMILY DWELLINGS,1,2904,22,NaN,B9,454 EAST 172 STREET,NaN,10457.0,2.0,0.0,2.0,"1,658",1428.0,1901.0,1,B9,350000


### SPLIT DATA

In [64]:
## Wrangle and analyse data
target  = 'SALE PRICE'
y = df[target]
X = df.drop(columns=target)
X.shape


(71833, 19)

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)